(1)

In [52]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapz

def monte_carlo_integration(n_samples):
    # Generate random samples uniformly distributed in [0, 1]
    samples = np.random.uniform(0, 1, n_samples)

    # Evaluate the function e^x at each sample point
    function_values = np.exp(samples)

    # Multiply the function values by the width of the interval and sum them up
    point_estimator = np.sum(function_values)

    # Calculate the variance of the estimator
    variance = (n_samples - 1) * np.mean(function_values ** 2) / np.exp(np.mean(samples))

    # Normalize the point estimator to obtain the true value of the integral
    integral_estimate = point_estimator / np.exp(np.mean(samples))

    return integral_estimate, variance

# Estimate the integral using 100 samples
n_samples = 10
estimate, variance = monte_carlo_integration(n_samples)
print(f"Estimated integral: {estimate}, Variance: {variance}")

# Construct a 95% confidence interval
confidence_level = 0.95
alpha = 1 - confidence_level
z = 1.96  # Standard normal deviate corresponding to the given confidence level

confidence_interval = (estimate - z * np.sqrt(variance), estimate + z * np.sqrt(variance))
print(f"95% Confidence interval: [{confidence_interval[0]}, {confidence_interval[1]}]")

Estimated integral: 10.155062368515146, Variance: 17.689055522841468
95% Confidence interval: [1.9116240266248035, 18.39850071040549]


(2)

In [35]:
def f(x):
    return np.exp(x)

n = 1000  # Number of samples
u = np.random.rand(n)
antithetic_u = np.concatenate((u[:n // 2], 1 - u[:n // 2]))

x = antithetic_u
y = f(x)
integral_estimate = np.mean(y)

integral_estimate

1.717341001328949

(3)

In [43]:
# Define the function to integrate
def f(x):
    return np.exp(x)

# Define the bounds of the integral
a = 0
b = 1

# Define the number of simulations
n = 100000

# Generate n random variables u from a uniform distribution on the interval [a, b]
u = np.random.uniform(a, b, n)

# Calculate the expected value of f(x) over the interval [a, b]
E_f = trapz(f(np.linspace(a, b, 1000)), np.linspace(a, b, 1000))

# Define a control variable
def g(x):
    return x

# Calculate the expected value of g(x) over the interval [a, b]
E_g = trapz(g(np.linspace(a, b, 1000)), np.linspace(a, b, 1000))

# Calculate the covariance between f(x) and g(x)
cov_fg = np.cov(f(u), g(u))[0][1]

# Calculate the variance of g(x)
var_g = np.var(g(u))

# Calculate the optimal value of the control variable
c = - cov_fg / var_g

# Calculate the estimated value of the integral
I = (1/n) * np.sum(f(u) + c * (g(u) - E_g))

# Print the estimated value of the integral
print("Estimated value of the integral:", I)

Estimated value of the integral: 1.7181098937339856


(4)

Integral estimate: 0.001


(5)

In [48]:
import numpy as np

def f(x):
    return np.exp(x)

strata = [(0, 0.5), (0.5, 1)]


def generate_samples(stratum, n):
    a, b = stratum
    return (b - a) * np.random.random(n) + a

n = 1000
samples = np.concatenate([generate_samples(stratum, n // len(strata)) for stratum in strata])


sample_mean = np.mean(f(samples))
sample_var = np.var(f(samples))

def g(x):
    return x - 1

cov = np.cov(f(samples), g(samples))[0, 1]

beta_star = cov / sample_var

control_variate_mean = sample_mean - beta_star * (np.mean(g(samples)) - g(0.5))

print("Control variate estimate:", control_variate_mean)
print("True value:", np.exp(1) - 1)





Control variate estimate: 1.724717579635188
True value: 1.718281828459045
